# Metrics II Part 2 PS 3
$$A(\theta)X_t = B(\theta)\mathbb{E}\left[X_{t+1} \right] + C(\theta)X_{t-1} + D(\theta)\eta_t$$ with
$$X_t = \left[x_t \, \pi_t \, i_t \, g_t \, u_t\right]'$$ $$\eta_t = \left[0 \, 0 \, \epsilon_{it} \, \epsilon_{gt} \, \epsilon_{ut} \right]'$$
## (a)
Coding up the system in matrices:

In [30]:
σ = 1; κ = 0.2; β = 0.99; ϕ_π = 1.25; ϕ_x = 0.25; ρ_i = 0.5; 
σ_i = 2; ρ_g = 0.75; ρ_u = 0.75; σ_g = 1; σ_u = 1; σ_m = 0.1;

A = [1 0 σ -1 0; -κ 1 0 0 -1; -ϕ_x*(1-ρ_i) -ϕ_π*(1-ρ_i) 1 0 0; 0 0 0 1 0; 0 0 0 0 1]
B = [1 σ 0 0 0; 0 β 0 0 0; 0 0 0 0 0; 0 0 0 0 0; 0 0 0 0 0]
C = [0 0 0 0 0; 0 0 0 0 0; 0 0 ρ_i 0 0; 0 0 0 ρ_g 0; 0 0 0 0 ρ_u]
D = [0 0 0 0 0; 0 0 0 0 0; 0 0 σ_i 0 0; 0 0 0 σ_g 0; 0 0 0 0 σ_u];

## (b)
Solving for $F$ and $G$ in our guess from the lecture notes
$$X_t = F(\theta)X_{t-1} + G(\theta)\eta_t$$but I bypass the Schur decomposition and find $F$ numerically:

In [35]:
using Optim

f = optimize(f -> norm(B*reshape(f, 5, 5)^2 - A*reshape(f, 5, 5) + C),  reshape(eye(5),25), 
    BFGS(), Optim.Options(g_tol = 1e-12))
F = reshape(f.minimizer, 5, 5)

5×5 Array{Float64,2}:
  1.40737      -0.200788     0.313952     -0.718155      1.63127    
 -0.442304      0.0631031    0.802762     -0.329794     -1.73695    
 -0.100519      0.014341     1.04097      -0.295891     -0.881684   
 -6.38151e-10  -2.23535e-9   4.09e-10      0.75         -2.09153e-10
  2.90496e-11  -3.19405e-9  -8.59732e-10  -3.32865e-10   0.75       

In [102]:
QZ = schurfact(vcat(hcat(zeros(5,5), eye(5)), hcat(-C, A)), 
    vcat(hcat(eye(5), zeros(5,5)), hcat(zeros(5,5), B))) 

Base.LinAlg.GeneralizedSchur{Float64,Array{Float64,2}}([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; -0.0 -0.0 … 0.6 0.8; -0.0 -0.0 … 0.0 1.25],[1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.8 -0.6; 0.0 0.0 … 0.0 0.0],Complex{Float64}[0.0+0.0im,0.0+0.0im,0.337617+0.0im,0.897735+0.0im,1.28184+0.0im,1.28695+0.0im,0.6+0.0im,-1.25+0.0im,0.6+0.0im,1.25+0.0im],[1.0,1.0,1.03666,0.633462,1.17143,0.0,0.8,0.0,0.8,0.0],[1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 1.0],[1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.6 0.8])

In [103]:
QZ.Q[1:5,1:5]*QZ.T[1:5,1:5]*inv(QZ.S[1:5,1:5])*inv(QZ.Q[1:5,1:5])

LoadError: Base.LinAlg.LAPACKException(1)

In [104]:
select = real(QZ.alpha) .<= 0.6
select[9] = false
ordschur(QZ, select)

Base.LinAlg.GeneralizedSchur{Float64,Array{Float64,2}}([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; -0.0 -0.0 … 0.6 0.8; -0.0 -0.0 … 0.0 1.25],[1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.8 -0.6; 0.0 0.0 … 0.0 0.0],Complex{Float64}[0.0+0.0im,0.0+0.0im,0.337617+0.0im,0.740712+0.0im,-1.78599+0.0im,0.88503+0.0im,0.737155+0.0im,1.28695+0.0im,0.6+0.0im,1.25+0.0im],[1.0,1.0,1.03666,0.987615,4.46761e-17,0.624496,0.67366,0.0,0.8,0.0],[1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 1.0],[1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.6 0.8])

In [105]:
QZ.Z[6:end,1:5]*QZ.Z[1:5,1:5]^-1

LoadError: Base.LinAlg.LAPACKException(4)

In [106]:
QZ.Q[1:5,1:5]*QZ.T[1:5,1:5]*inv(QZ.S[1:5,1:5])*inv(QZ.Q[1:5,1:5])

LoadError: Base.LinAlg.LAPACKException(1)

In [36]:
G = inv(A-B*F)*D

5×5 Array{Float64,2}:
 0.0  0.0  3.20279  -1.55253   -3.33805 
 0.0  0.0  2.59916  -0.252734  -0.583295
 0.0  0.0  4.02482  -0.352025  -0.781816
 0.0  0.0  0.0       1.0        0.0     
 0.0  0.0  0.0       0.0        1.0     

Observables:
$$Y_t = \left[x_t^{obs} \, \pi_t^{obs} \, i_t^{obs} \right]$$ with
$$Y_t = H(\theta)X_t + J(\theta)\nu_t$$ and:

In [32]:
H = [1 0 0 0 0; 0 1 0 0 0; 0 0 1 0 0]
J = σ_m;

## (c)

In [67]:
using Distributions

N = 1000
η = rand(MvNormal(zeros(5), eye(5)), N)
ν = rand(MvNormal(zeros(3), eye(3)), N)
X = zeros(5,N)
Y = zeros(3,N)

for i in 2:size(X)[2]
    X[:,i] = F*X[:,i-1] + G*η[:,i]
    Y[:,i] = H*X[:,i] + J*ν[:,i]
end

In [69]:
X[4,:]

1000-element Array{Float64,1}:
  0.0        
 -0.81663    
  0.446598   
 -0.148844   
 -1.64879    
  1.06498    
  1.83038    
  2.82678    
  2.29844    
  2.00653    
  1.79416    
  1.35265    
  1.50242    
  ⋮          
  2.1101e140 
  2.99042e140
  4.23799e140
  6.00604e140
  8.5117e140 
  1.20627e141
  1.70951e141
  2.42271e141
  3.43344e141
  4.86583e141
  6.89581e141
  9.77267e141